In [ ]:
# Install packages
!pip install transformers textattack datasets --quiet

# Imports
import os, random, numpy as np, torch
from copy import deepcopy
import nltk
nltk.download('averaged_perceptron_tagger_eng')

from transformers import AutoModelForSequenceClassification, AutoTokenizer
from textattack.models.wrappers import HuggingFaceModelWrapper
from textattack.datasets import HuggingFaceDataset
from textattack import Attacker, AttackArgs

# Import recipes - fixed import paths
from textattack.attack_recipes import (
    TextFoolerJin2019,
    DeepWordBugGao2018,
    PWWSRen2019,
    BAEGarg2019,
)

# Set seed
seed = 42
os.environ['PYTHONHASHSEED'] = str(seed)
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Running on device:", device)

# Load & wrap model
model_name = "textattack/bert-base-uncased-imdb"
model = AutoModelForSequenceClassification.from_pretrained(model_name).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)
wrapper = HuggingFaceModelWrapper(model, tokenizer)

# Dataset - limit to smaller subset for testing
dataset = HuggingFaceDataset("imdb", split="test")

# Attack arguments
base_args = AttackArgs(
    num_examples=5,  # Reduced for faster testing
    random_seed=seed,
    shuffle=False,
    disable_stdout=False,  # Enable to see progress
    log_to_csv=None,
    parallel=False  # Disable parallel processing to avoid issues
)

# Define attacks - fixed class references
attacks = {
    "TextFooler": TextFoolerJin2019.build(wrapper),
    "DeepWordBug": DeepWordBugGao2018.build(wrapper),
    "PWWS": PWWSRen2019.build(wrapper),
    "BAE": BAEGarg2019.build(wrapper),
}

# Run attacks with error handling and CSV saving
results = {}
for name, attack in attacks.items():
    print(f"Running {name}...")
    try:
        args = deepcopy(base_args)
        # Set CSV filename for this attack
        csv_filename = f"{name.replace(' ', '_').replace('(', '').replace(')', '')}_results.csv"
        args.log_to_csv = csv_filename

        attacker = Attacker(attack, dataset, args)
        result = attacker.attack_dataset()
        results[name] = result
        print(f"{name} completed successfully — results saved to {csv_filename}\n")
    except Exception as e:
        print(f"Error running {name}: {str(e)}")
        print(f"Skipping {name} and continuing...\n")
        continue

print("All attacks completed!")
print("Results summary:")
for name, result in results.items():
    if result:
        print(f"{name}: {len(result)} examples processed")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 126.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


Running on device: cuda


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
textattack: Loading datasets dataset imdb, split test.
textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
If you want to use `RobertaLMHeadModel` as a standalone, add `is_decoder=True.`
textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSeque

Running CheckList...
Attack(
  (search_method): GreedySearch
  (goal_function):  UntargetedClassification
  (transformation):  CompositeTransformation(
    (0): WordSwapExtend
    (1): WordSwapContract
    (2): WordSwapChangeName
    (3): WordSwapChangeNumber
    (4): WordSwapChangeLocation
    )
  (constraints): 
    (0): RepeatModification
  (is_black_box):  True
) 



...

--------------------------------------------- Result 2 ---------------------------------------------
[[Negative (100%)]] --> [[Positive (67%)]]

Worth the entertainment value of a rental, especially if you like action movies. This one features the usual car chases, fights with the great Van Damme kick style, shooting battles with the 40 shell load shotgun, and even terrorist style bombs. All of this is [[entertaining]] and competently handled but there is nothing that really blows you away if you've seen your share before.<br /><br />The plot is made interesting by the inclusion of a rabbit, which is clever but hardly profound. Many of the characters are [[heavily]] stereotyped -- the angry veterans, the terrified illegal aliens, the crooked cops, the indifferent feds, the bitchy [[tough]] lady station head, the crooked politician, the fat federale who looks like he was typecast as the Mexican in a Hollywood movie from the 1940s. All passably acted but again [[nothing]] special.<br /><

[Succeeded / Failed / Skipped / Total] 2 / 1 / 0 / 3:  60%|██████    | 3/5 [22:31<15:01, 450.50s/it]

--------------------------------------------- Result 3 ---------------------------------------------
[[Negative (100%)]] --> [[Positive (77%)]]

its a [[totally]] average film with a few semi-alright action sequences that make the plot seem a little better and remind the viewer of the classic van dam films. parts of the plot don't make sense and seem to be added in to use up time. the [[end]] plot is that of a very basic type that doesn't leave the viewer [[guessing]] and any twists are obvious from the beginning. the end scene with the [[flask]] backs don't make sense as they are added in and seem to have little relevance to the history of van dam's character. not [[really]] worth watching again, bit disappointed in the end production, even though it is apparent it was shot on a low budget certain shots and sections in the [[film]] are of poor directed quality

its a [[absolutely]] average film with a few semi-alright action sequences that make the plot seem a little better and remind

[Succeeded / Failed / Skipped / Total] 3 / 1 / 1 / 5: 100%|██████████| 5/5 [40:18<00:00, 483.78s/it]

--------------------------------------------- Result 4 ---------------------------------------------
[[Negative (100%)]] --> [[Positive (93%)]]

STAR RATING: ***** Saturday Night **** [[Friday]] Night *** Friday Morning ** Sunday Night * Monday Morning <br /><br />[[Former]] New Orleans homicide [[cop]] Jack Robideaux (Jean Claude Van Damme) is re-assigned to Columbus, a small but violent town in Mexico to help the police there with their efforts to stop a major [[heroin]] smuggling [[operation]] into their town. The [[culprits]] turn out to be ex-military, lead by former [[commander]] Benjamin Meyers (Stephen Lord, otherwise known as Jase from East Enders) who is using a special [[method]] he learned in Afghanistan to fight off his [[opponents]]. But Jack has a more personal reason for taking him down, that draws the two [[men]] into an explosive [[final]] showdown where only one will [[walk]] away [[alive]].<br /><br />After Until [[Death]], Van Damme appeared to be on a high, [[show

In [ ]:
import os

# List all CSV files in the current directory
[file for file in os.listdir() if file.endswith(".csv")]


['Alzantot_results.csv', 'CheckList_results.csv']

In [ ]:
from google.colab import files
files.download('TextFooler_results.csv')
files.download('DeepWordBug_results.csv')
files.download('PWWS_results.csv')
files.download('BAE_results.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#Select your csv files
from google.colab import files

uploaded = files.upload()

Saving BAE_results.csv to BAE_results.csv
Saving DeepWordBug_results.csv to DeepWordBug_results.csv
Saving PWWS_results.csv to PWWS_results.csv
Saving TextFooler_results.csv to TextFooler_results.csv


In [ ]:
# Install required packages
!pip install -q sentence-transformers transformers

# Imports
from sentence_transformers import SentenceTransformer, util
from transformers import GPT2LMHeadModel, GPT2TokenizerFast
import torch
import pandas as pd

# Load models
semantic_model = SentenceTransformer('all-MiniLM-L6-v2')
gpt2_tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
gpt2_model = GPT2LMHeadModel.from_pretrained("gpt2")
gpt2_model.eval()

# Load CSVs
textfooler = pd.read_csv("TextFooler_results.csv")
deepwordbug = pd.read_csv("DeepWordBug_results.csv")
pwws = pd.read_csv("PWWS_results.csv")
bae = pd.read_csv("BAE_results.csv")

methods = {
    "TextFooler": textfooler,
    "DeepWordBug": deepwordbug,
    "PWWS": pwws,
    "BAE": bae

}

# --- Metric Functions ---

def success_rate(df):
    df = df[df['result_type'] != 'Skipped']
    return (df['result_type'] == 'Successful').mean()

def avg_words_perturbed(df, result_type):
    filtered = df[df['result_type'] != 'Skipped']
    if result_type == 'Successful':
        filtered = filtered[filtered['result_type'] == 'Successful']
    elif result_type == 'Failed':
        filtered = filtered[filtered['result_type'] != 'Successful']
    counts = filtered['perturbed_text'].apply(lambda x: str(x).count('[['))
    return counts.mean()

def semantic_similarity(original_texts, perturbed_texts):
    embeddings1 = semantic_model.encode(original_texts.tolist(), convert_to_tensor=True)
    embeddings2 = semantic_model.encode(perturbed_texts.tolist(), convert_to_tensor=True)
    similarities = util.cos_sim(embeddings1, embeddings2)
    return similarities.diag().cpu().numpy().mean()

def avg_fluency_score(texts):
    scores = []
    for t in texts:
        encodings = gpt2_tokenizer(str(t), return_tensors='pt')
        with torch.no_grad():
            outputs = gpt2_model(**encodings, labels=encodings["input_ids"])
            log_likelihood = outputs.loss.item()
            scores.append(-log_likelihood)  # Higher is better (less perplexity)
    return sum(scores) / len(scores)

# --- Final Output ---

for name, df in methods.items():
    df = df[df['result_type'] != 'Skipped']
    success_df = df[df['result_type'] == 'Successful']
    fail_df = df[df['result_type'] != 'Successful']

    sr = success_rate(df) * 100
    avg_success = avg_words_perturbed(df, 'Successful')
    avg_fail = avg_words_perturbed(df, 'Failed')

    if not success_df.empty:
        sim_score = semantic_similarity(success_df['original_text'], success_df['perturbed_text'])
        fluency_score = avg_fluency_score(success_df['perturbed_text'])
    else:
        sim_score = float('nan')
        fluency_score = float('nan')

    print(f"{name} Success Rate: {sr:.2f}%")
    print(f"{name} Avg. Words Perturbed (Successes): {avg_success:.2f}")
    print(f"{name} Avg. Words Perturbed (Fails): {avg_fail:.2f}")
    print(f"{name} Avg. Semantic Similarity (Successes): {sim_score:.3f}")
    print(f"{name} Avg. Fluency Score (GPT-2) (Successes): {fluency_score:.2f}")
    print("---")


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


TextFooler Success Rate: 100.00%
TextFooler Avg. Words Perturbed (Successes): 19.72
TextFooler Avg. Words Perturbed (Fails): nan
TextFooler Avg. Semantic Similarity (Successes): 0.951
TextFooler Avg. Fluency Score (GPT-2) (Successes): -3.95
---
DeepWordBug Success Rate: 44.44%
DeepWordBug Avg. Words Perturbed (Successes): 10.62
DeepWordBug Avg. Words Perturbed (Fails): 25.00
DeepWordBug Avg. Semantic Similarity (Successes): 0.931
DeepWordBug Avg. Fluency Score (GPT-2) (Successes): -4.17
---
PWWS Success Rate: 100.00%
PWWS Avg. Words Perturbed (Successes): 13.50
PWWS Avg. Words Perturbed (Fails): nan
PWWS Avg. Semantic Similarity (Successes): 0.958
PWWS Avg. Fluency Score (GPT-2) (Successes): -3.90
---
BAE Success Rate: 72.22%
BAE Avg. Words Perturbed (Successes): 8.00
BAE Avg. Words Perturbed (Fails): 18.80
BAE Avg. Semantic Similarity (Successes): 0.989
BAE Avg. Fluency Score (GPT-2) (Successes): -3.86
---
